In [ ]:
# This makes a cube of FITS files, so as to inspect images en masse with DS9

# Saved as .py file, 2017 Nov 19

# created 2017 Nov 19 by E.S.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas as pd
import datetime
from astropy.io import fits
import asciitable
from pathlib import Path

In [2]:
# set file paths and datasets

dirTreeStem = ('/home/../../media/unasemaje/Seagate Expansion Drive/lbti_data_reduction/'
               'lmircam_astrometry/ut_2017_11_06/asterism/processedData/')
retrievalPiece = ('step03_derotate/')
depositPiece = ('step04_ditherMedians/')
fileNameStem = ('lm_171106_')

In [3]:
def make_median(ditherPos,startNum,stopNum,xSize=2048,ySize=2048,N=1):
    '''
    makes a median from a range of FITS files
    INPUTS--
    ditherPos: number indicating the dither number
    startNum: first frame number
    stopNum: last frame number (inclusive)
    xSize: pixels in x-dimension
    ySize: pixels in y-dimension
    N: number of frames between each one used to make the median (default is 1: no skipped frames)
    '''
    N = N # use every N frames
    xSize = xSize
    ySize = ySize
    sliceNum = 0 # initialize cube slice number
    cubeArray = np.zeros((int(np.divide(stopNum-startNum+1,N)),xSize,ySize),dtype=np.int64)
    for f in range(startNum,stopNum+1): # loop over filenames
        if (f%N == 0): 
            print('----------------------------------------')
            print(str(f))
            image, header = fits.getdata(dirTreeStem+
                                         retrievalPiece+
                                 fileNameStem+
                                 str("{:0>5d}".format(f))+
                                 '.fits',
                                 0,
                                 header=True)
            
            cubeArray[sliceNum,:,:] = image
            sliceNum += 1 # move to next slice
    
    # take median
    medianArray = np.median(cubeArray,axis=0)
    
    # save median
    hdu = fits.PrimaryHDU(medianArray)
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(dirTreeStem+depositPiece+fileNameStem+'dither_'+str("{:0>2d}".format(ditherPos))+'.fits',
                    overwrite=False)

In [20]:
# run the function

make_median(16,8923,8941)

----------------------------------------
8923
----------------------------------------
8924
----------------------------------------
8925
----------------------------------------
8926
----------------------------------------
8927
----------------------------------------
8928
----------------------------------------
8929
----------------------------------------
8930
----------------------------------------
8931
----------------------------------------
8932
----------------------------------------
8933
----------------------------------------
8934
----------------------------------------
8935
----------------------------------------
8936
----------------------------------------
8937
----------------------------------------
8938
----------------------------------------
8939
----------------------------------------
8940
----------------------------------------
8941
